In [66]:
import numpy as np
from scipy.optimize import linprog
#import pandas as pd
import time
from math import floor, log
import time
import csv
import json
from sage.matrix.all import Matrix
from sage.numerical.mip import MixedIntegerLinearProgram
from sage.numerical.mip import MIPSolverException
#import sage.matrix.constructor as matrix

# generate the ph_table from logic m
def generate_p_matrix(m, p_matrix):
    acc_prod = list(map(lambda x: int(x), [np.prod(m[i+1:]) for i in range(len(m))]))
    l = np.sum(m)-1
    p = [() for _ in range(np.prod(m))]
    level = []

    for i in range(len(p)):
        total = i
        level = []
        for j in range(len(m)):
            current = int(total/acc_prod[j])
            total %= acc_prod[j]
            level.append(current)
        curr_p = []
        for k in range(len(level)):
            temp = [0 for _ in range(m[k])]
            temp[m[k]-1-level[k]] = 1
            curr_p+=temp
        p_matrix.append(curr_p)
        #print(p_matrix)
        
# generate the initial set of basis for the cone for delta
def solvePLL(m, y):
    
    p = MixedIntegerLinearProgram(maximization=True, solver='ppl')
    x = p.new_variable(nonnegative=True)
    p.set_objective(0)
    p.add_constraint(m * x <= y)
    
    try:
        p.solve()
        return True
    except MIPSolverException as err:
        s = str(err)
        if "no feasible solution" in s:
            return False
    return False

def generate_basis_matrix(m, p_matrix, lhs):
    basis_matrix = []
    acc_prod = list(map(lambda x: int(x), [np.prod(m[i+1:]) for i in range(len(m))]))
    for i in range(len(m)):

        basis = m[i] - 1
        while basis:

            temp = basis
            count=0
            while temp:
                count+=1
                temp>>=1

            for j in range(count):
                if (basis & (1<<j)):
                    p1 = basis*acc_prod[i]
                    p2 = (basis.__xor__(1<<j))*acc_prod[i]
                    print(p1,p2)
                    basis_matrix.append(p_matrix[p1]-p_matrix[p2])
                    lhs.add((p1,p2))

            basis -=1

    return  np.array(basis_matrix) 

def generate_relations(relations, extra_matrix, p_matrix, permutations):
    p = p_matrix
    
    l_h = []
    
    for i,j in relations:
        current = (p_matrix[i] - p_matrix[j]).reshape((1,-1))
        l_h.append(current)
        
    basis_matrix = np.concatenate(l_h,axis =0)
    if len(extra_matrix) > 0:
        basis_matrix = np.concatenate([basis_matrix, extra_matrix],axis = 0)
        basis_matrix = np.unique(basis_matrix, axis = 0)
        
    A_eq = basis_matrix.T
    #b_ub = np.array([0 for _ in range(len(basis_matrix))])    
    #c = np.array([0 for _ in range(len(basis_matrix))])
    #A_ub = - np.identity(len(basis_matrix))
    
    ret = set()
    # print(basis_matrix)
    for i,j in permutations:
        #print(i,j)
        m = Matrix(A_eq)
        y = (p[i]-p[j]).tolist()
        solvable = solvePLL(m,y)
        if solvable:
            ret.add((i,j))
    return ret

def get_permutations(n):
    ret = []
    for i in range(n):
        for j in range(n):
            if i!=j:
                ret.append((i,j))
    return ret


def test_in_relations(relations, extra_matrix, p_matrix, candidate):
    p = p_matrix
    l_h = []
    
    for i,j in relations:
        current = (p_matrix[i] - p_matrix[j]).reshape((1,-1))
        l_h.append(current)
    
    
    basis_matrix = np.concatenate(l_h,axis =0)
    if len(extra_matrix)>0:
        basis_matrix = np.concatenate([basis_matrix, extra_matrix],axis = 0)
        
    A_eq = basis_matrix.T
    #b_ub = np.array([0 for _ in range(len(basis_matrix))])    
    #c = np.array([0 for _ in range(len(basis_matrix))])
    #A_ub = - np.identity(len(basis_matrix))
    
    i, j = candidate[0], candidate[1]
    
    m = Matrix(A_eq)
    y = (p[i]-p[j]).tolist()
    solvable = solvePLL(m,y)
    
    return solvable

    #if linprog(c = c,A_ub=A_ub,A_eq=A_eq,b_ub = b_ub,b_eq=p[i]-p[j]).success:
    #    return True
    #else:
    #    return False
    

def generate_predecessors(relations, extra_matrix, p_matrix, permutations):
    zero_logic = generate_relations(relations, extra_matrix, p_matrix, permutations)
    ret = dict()
    for i, j in zero_logic:
        if i in ret:
            ret[i].add(j)
        else:
            ret[i] = set([j])
    return ret

def generate_logics(basic_relations, extra_matrix, p_matrix, permutations):
    ret = dict()
    basic_set = set()
    for i, j in permutations:
        if test_in_relations(basic_relations,extra_matrix,p_matrix,(i,j)):
            basic_set.add((i,j))
    
    for i, j in permutations:
        if ((i, j) not in basic_set) and ((j,i) not in basic_set) :
            temp_set = set()
            relations = basic_relations.union(set([(i,j)]))
            #print(relations)
            for k,l in permutations:
                if (k,l)!=(i,j) and (not (k,l) in basic_set) and \
                test_in_relations(relations,extra_matrix,p_matrix,(k,l)):
                    if (i,j) in ret:
                        ret[(i,j)].add((k,l))
                    else:
                        ret[(i,j)] = set([(k,l)])
    return ret


   
def indexlist(index,pattern):
    res = []
    for i in range(len(pattern)-1,-1,-1):
        res.append(index%pattern[i])
        index= floor(index/pattern[i])
    res = res[::-1]
    return res


# pattern = [4,2,2]
# index = 4

# indexlist(index, pattern)

# sample = [[X_0, X_1, X_2,X_3],[Y_0,Y_1],[Z_0,Z_1]]
def evalutionord(pattern, sample):
    count  = 1
    for i in range(len(pattern)):
        count*=pattern[i]
    ord = []
    for i in range(count):
        ilist = indexlist(i, pattern)
        x, y, z = sample[0][ilist[0]], sample[1][ilist[1]], sample[2][ilist[2]]
        ord.append(x+y+z)
    return np.argsort(ord)


def random_sample(pattern):
    sample = []
    for i in range(len(pattern)):
        temp = []
        count = floor(log(pattern[i],2))
        points = np.random.uniform(size = count+1)
        first = points[0]
        points[0] = -1
        points = sorted(points)
        points[0] = first
        for j in range(pattern[i]):
            p = j
            pos = 0
            current = points[0]
            while p:
                #print(p, pos)
                v = p%2
                if v:
                    current += points[pos+1]
                pos+=1
                p = floor(p/2)
            temp.append(current)
        sample.append(list(map(lambda x: log(x),temp)))
    return sample


def random_ordering(pattern, time_limit = 1000, size_limpoit = 1000):
    ret = []
    d = {}
    count = time_limit
    for i in range(count):
        sample = random_sample(pattern)
        ord = tuple(evalutionord(pattern,sample))
        ret.append(ord)
        if ord not in d:
            d[ord] = sample
    return ret, d

def extraRelationGenerator(p_matrix, ptotalordering):
    ret = []
    for i in range(1,len(ptotalordering)):
        ret.append(p_matrix[ptotalordering[i]] - p_matrix[ptotalordering[i-1]])
    return np.vstack(ret)


def run5rational(logic, restrictedOrdering, extraFile):
    #count = 0
    ret = set([])
    file_name = '{} {}.dat'.format(logic,restrictedOrdering)
    # set parameters
    m =  list(map(lambda x: int(x), logic.split('_')))
    n = np.prod(m)

    # generate p_matrix
    p_matrix = []
    generate_p_matrix(m,p_matrix)
    p_matrix = np.array(p_matrix)

    # get original basis
    original_basis = set([])
    basis_matrix = generate_basis_matrix(m, p_matrix, original_basis)

    # get basis matrix not that useful
    # also delta 1 < delta 2
    extra_matrix = extraRelationGenerator(p_matrix,restrictedOrdering)
    #extra_matrix = np.vstack([extra_matrix, np.array([[0,1,-1,0,0,0,0,0,0,0],[-1,1,1,-1,0,0,0,0,0,0]])])
    #tt = [[-1, 0, 0, 1, 1, 0, 0, -1, 0, 0], [-1, 0, 1, 0, 0, 1, 0, -1, 0, 0], [-1, 0, 1, 0, 1, 0, -1, 0, 0, 0], [-1, 1, 0, 0, 0, 0, 1, -1, 0, 0], [-1, 1, 0, 0, 1, -1, 0, 0, 0, 0], [-1, 1, 1, -1, 0, 0, 0, 0, 0, 0], [0, -1, 0, 1, 0, 1, 0, -1, 0, 0], [0, 0, -1, 1, 0, 0, 1, -1, 0, 0], [0, 0, 0, 0, -1, 1, 1, -1, 0, 0]]
    #tt = np.array(tt)
    with open(extraFile,'r') as file:
        tt = np.array(json.load(file))
 
    extra_matrix = np.vstack([extra_matrix, tt])
    
    # basis_matrix = np.concatenate([basis_matrix,extra_matrix],axis = 0)
    # permutations
    permutations  = get_permutations(n)

    # get permutations we will loop in algoritm
    candidates = set(range(1,n))

    # predecessors
    predecessors = generate_predecessors(original_basis, extra_matrix, p_matrix, permutations)

    print('predecessors done')
    # first_order_logics
    logics = generate_logics(original_basis, extra_matrix, p_matrix, permutations)
    print('logics done')

    start = time.time()
    path= [0]
    trie_dfs(original_basis, set([]), p_matrix, extra_matrix, candidates, path, ret,permutations,predecessors,logics,file_name)
    #time.time() - start

    #with open(file_name,'a') as file:
    #    file.write(str(time.time()-start)+'\n')

def run3rational(logic,filename,extraFile=None):
    """filename: output filename"""
    #count = 0
    ret = set([])
    #file_name = '{}.dat'.format(logic)
    # set parameters
    m =  list(map(lambda x: int(x), logic.split('_')))
    n = np.prod(m)

    # generate p_matrix
    p_matrix = []
    generate_p_matrix(m,p_matrix)
    p_matrix = np.array(p_matrix)

    # get original basis
    original_basis = set([])
    #basis_matrix = generate_basis_matrix(m, p_matrix, original_basis)
    generate_basis_matrix(m, p_matrix, original_basis)
    #extra_matrix = None
    # get basis matrix not that useful
    # also delta 1 < delta 2
    #extra_matrix = extraRelationGenerator(p_matrix,restrictedOrdering)
    #extra_matrix = np.vstack([extra_matrix, np.array([[0,1,-1,0,0,0,0,0,0,0],[-1,1,1,-1,0,0,0,0,0,0]])])
    #tt = [[-1, 0, 0, 1, 1, 0, 0, -1, 0, 0], [-1, 0, 1, 0, 0, 1, 0, -1, 0, 0], [-1, 0, 1, 0, 1, 0, -1, 0, 0, 0], [-1, 1, 0, 0, 0, 0, 1, -1, 0, 0], [-1, 1, 0, 0, 1, -1, 0, 0, 0, 0], [-1, 1, 1, -1, 0, 0, 0, 0, 0, 0], [0, -1, 0, 1, 0, 1, 0, -1, 0, 0], [0, 0, -1, 1, 0, 0, 1, -1, 0, 0], [0, 0, 0, 0, -1, 1, 1, -1, 0, 0]]
    #tt = np.array(tt)
    if extraFile:
        with open(extraFile,'r') as file:
            tt = np.array(json.load(file))
        extra_matrix = tt
        #basis_matrix = np.concatenate([basis_matrix,extra_matrix],axis = 0)
    else:
        extra_matrix = np.array([])
        
    # permutations
    permutations  = get_permutations(n)

    # get permutations we will loop in algoritm
    candidates = set(range(1,n))

    # predecessors
    predecessors = generate_predecessors(original_basis, extra_matrix, p_matrix, permutations)

    print('predecessors done')
    # first_order_logics
    logics = generate_logics(original_basis, extra_matrix, p_matrix, permutations)
    print('logics done')

    start = time.time()
    path= [0]
    trie_dfs(original_basis, set([]), p_matrix, extra_matrix, candidates, path, ret,permutations,predecessors,logics,filename)
    #time.time() - start

    #with open(file_name,'a') as file:
    #    file.write(str(time.time()-start)+'\n')

In [67]:
def trie_dfs(original_basis, newly_added, p_matrix, extra_matrix, candidates, path, ret, permutations, predecessors, logics, file_name):
    print("The table of predecessors \n : ", predecessors)
    if not candidates:
        temp_rel = generate_relations(original_basis.union(newly_added), extra_matrix, p_matrix, permutations)
        
        print("End")
        if len(temp_rel) > int(len(permutations)/2):
            return
        
        
        with open(file_name,'a') as file:
            print("Collect the result")
            file.write(str(path)+'\n')

        ret.add(tuple(path))
    
    temp_settled = set(path)
    
    for i in candidates:
        
        print("current path : ", path)
        print("start the next candidate : ", i)
        if not predecessors[i].issubset(temp_settled):
            print("There is a predecessor not have been settled continue : ", i)
            continue
        j  = path[-1]
        
        # if inverse in the set then we just continue, else we can add i safely
        if test_in_relations(original_basis.union(newly_added), extra_matrix, p_matrix, (j, i)):
            print("Can not be added to the last by reverse relation : ", (j,i))
            continue
        else:
            print("No reverse relation add to the last by : ", (i))
            newly_added.add((i,j))
            path.append(i)
            candidates.remove(i)
            temp_predecessors = dict()
            for key in predecessors:
                temp_predecessors[key] = predecessors[key].copy()
            if (i, j) in logics:
                for k,l in logics[(i,j)]:
                    temp_predecessors[k].add(l)
            trie_dfs(original_basis, newly_added, p_matrix, extra_matrix, candidates, path, ret, permutations,temp_predecessors,logics,file_name)
            newly_added.remove((i,j))
            path.pop()
            candidates.add(i)   
         

In [68]:
logic,filename,extraFile='4_2','42_symmetric.dat','../../../extraRestriction/restrictions21.json'

ret = set([])
#file_name = '{}.dat'.format(logic)
# set parameters

m =  list(map(lambda x: int(x), logic.split('_')))
n = np.prod(m)

# generate p_matrix
p_matrix = []
generate_p_matrix(m,p_matrix)
p_matrix = np.array(p_matrix)

# get original basis
original_basis = set([])
#basis_matrix = generate_basis_matrix(m, p_matrix, original_basis)
generate_basis_matrix(m, p_matrix, original_basis)
#extra_matrix = None
# get basis matrix not that useful
# also delta 1 < delta 2
#extra_matrix = extraRelationGenerator(p_matrix,restrictedOrdering)
#extra_matrix = np.vstack([extra_matrix, np.array([[0,1,-1,0,0,0,0,0,0,0],[-1,1,1,-1,0,0,0,0,0,0]])])
#tt = [[-1, 0, 0, 1, 1, 0, 0, -1, 0, 0], [-1, 0, 1, 0, 0, 1, 0, -1, 0, 0], [-1, 0, 1, 0, 1, 0, -1, 0, 0, 0], [-1, 1, 0, 0, 0, 0, 1, -1, 0, 0], [-1, 1, 0, 0, 1, -1, 0, 0, 0, 0], [-1, 1, 1, -1, 0, 0, 0, 0, 0, 0], [0, -1, 0, 1, 0, 1, 0, -1, 0, 0], [0, 0, -1, 1, 0, 0, 1, -1, 0, 0], [0, 0, 0, 0, -1, 1, 1, -1, 0, 0]]
#tt = np.array(tt)
if extraFile:
    with open(extraFile,'r') as file:
        tt = np.array(json.load(file))
    extra_matrix = tt
    #basis_matrix = np.concatenate([basis_matrix,extra_matrix],axis = 0)
else:
    extra_matrix = np.array([])

# permutations
permutations  = get_permutations(n)

# get permutations we will loop in algoritm
candidates = set(range(1,n))

# predecessors
predecessors = generate_predecessors(original_basis, extra_matrix, p_matrix, permutations)

print('predecessors done')
# first_order_logics
logics = generate_logics(original_basis, extra_matrix, p_matrix, permutations)
print('logics done')

start = time.time()
path= [0]
trie_dfs(original_basis, set([]), p_matrix, extra_matrix, candidates, path, ret,permutations,predecessors,logics,filename)
#time.time() - start

#with open(file_name,'a') as file:
#    file.write(str(time.time()-start)+'\n')

6 4
6 2
4 0
2 0
1 0
predecessors done
logics done
The table of predecessors 
 :  {6: {0, 2, 4}, 3: {0, 1, 2}, 5: {0, 1, 4}, 7: {0, 1, 2, 3, 4, 5, 6}, 2: {0}, 1: {0}, 4: {0}}
current path :  [0]
start the next candidate :  1
No reverse relation add to the last by :  1
The table of predecessors 
 :  {6: {0, 2, 4}, 3: {0, 1, 2}, 5: {0, 1, 4}, 7: {0, 1, 2, 3, 4, 5, 6}, 2: {0}, 1: {0}, 4: {0}}
current path :  [0, 1]
start the next candidate :  2
No reverse relation add to the last by :  2
The table of predecessors 
 :  {6: {0, 1, 2, 4}, 3: {0, 1, 2}, 5: {0, 1, 4}, 7: {0, 1, 2, 3, 4, 5, 6}, 2: {0}, 1: {0}, 4: {0}}
current path :  [0, 1, 2]
start the next candidate :  3
No reverse relation add to the last by :  3
The table of predecessors 
 :  {6: {0, 1, 2, 4}, 3: {0, 1, 2}, 5: {0, 1, 4}, 7: {0, 1, 2, 3, 4, 5, 6}, 2: {0}, 1: {0}, 4: {0}}
current path :  [0, 1, 2, 3]
start the next candidate :  4
No reverse relation add to the last by :  4
The table of predecessors 
 :  {6: {0, 1, 2, 3, 4}, 3:

End
Collect the result
current path :  [0, 1, 4, 2, 5, 3]
start the next candidate :  7
There is a predecessor not have been settled continue :  7
current path :  [0, 1, 4, 2, 5]
start the next candidate :  6
No reverse relation add to the last by :  6
The table of predecessors 
 :  {6: {0, 1, 2, 4}, 3: {0, 1, 2, 4, 5}, 5: {0, 1, 4}, 7: {0, 1, 2, 3, 4, 5, 6}, 2: {0, 1}, 1: {0}, 4: {0}}
current path :  [0, 1, 4, 2, 5, 6]
start the next candidate :  3
No reverse relation add to the last by :  3
The table of predecessors 
 :  {6: {0, 1, 2, 4}, 3: {0, 1, 2, 4, 5}, 5: {0, 1, 4}, 7: {0, 1, 2, 3, 4, 5, 6}, 2: {0, 1}, 1: {0}, 4: {0}}
current path :  [0, 1, 4, 2, 5, 6, 3]
start the next candidate :  7
No reverse relation add to the last by :  7
The table of predecessors 
 :  {6: {0, 1, 2, 4}, 3: {0, 1, 2, 4, 5}, 5: {0, 1, 4}, 7: {0, 1, 2, 3, 4, 5, 6}, 2: {0, 1}, 1: {0}, 4: {0}}
End
Collect the result
current path :  [0, 1, 4, 2, 5, 6]
start the next candidate :  7
There is a predecessor not hav

End
Collect the result
current path :  [0, 2, 1, 4, 3, 6]
start the next candidate :  7
There is a predecessor not have been settled continue :  7
current path :  [0, 2, 1, 4, 3]
start the next candidate :  7
There is a predecessor not have been settled continue :  7
current path :  [0, 2, 1, 4]
start the next candidate :  5
There is a predecessor not have been settled continue :  5
current path :  [0, 2, 1, 4]
start the next candidate :  6
No reverse relation add to the last by :  6
The table of predecessors 
 :  {6: {0, 1, 2, 4}, 3: {0, 1, 2}, 5: {0, 1, 2, 4, 6}, 7: {0, 1, 2, 3, 4, 5, 6}, 2: {0}, 1: {0}, 4: {0}}
current path :  [0, 2, 1, 4, 6]
start the next candidate :  3
No reverse relation add to the last by :  3
The table of predecessors 
 :  {6: {0, 1, 2, 4}, 3: {0, 1, 2, 4}, 5: {0, 1, 2, 4, 6}, 7: {0, 1, 2, 3, 4, 5, 6}, 2: {0}, 1: {0}, 4: {0}}
current path :  [0, 2, 1, 4, 6, 3]
start the next candidate :  5
No reverse relation add to the last by :  5
The table of predecessors 


End
Collect the result
current path :  [0, 4, 1, 2, 5, 6]
start the next candidate :  7
There is a predecessor not have been settled continue :  7
current path :  [0, 4, 1, 2, 5]
start the next candidate :  7
There is a predecessor not have been settled continue :  7
current path :  [0, 4, 1, 2]
start the next candidate :  6
No reverse relation add to the last by :  6
The table of predecessors 
 :  {6: {0, 1, 2, 4}, 3: {0, 1, 2, 4, 6}, 5: {0, 1, 4}, 7: {0, 1, 2, 3, 4, 5, 6}, 2: {0}, 1: {0}, 4: {0}}
current path :  [0, 4, 1, 2, 6]
start the next candidate :  3
No reverse relation add to the last by :  3
The table of predecessors 
 :  {6: {0, 1, 2, 4}, 3: {0, 1, 2, 4, 6}, 5: {0, 1, 4}, 7: {0, 1, 2, 3, 4, 5, 6}, 2: {0}, 1: {0}, 4: {0}}
current path :  [0, 4, 1, 2, 6, 3]
start the next candidate :  5
Can not be added to the last by reverse relation :  (3, 5)
current path :  [0, 4, 1, 2, 6, 3]
start the next candidate :  7
There is a predecessor not have been settled continue :  7
current p

End
Collect the result
current path :  [0, 4, 2, 6, 1, 5]
start the next candidate :  7
There is a predecessor not have been settled continue :  7
current path :  [0, 4, 2, 6, 1]
start the next candidate :  7
There is a predecessor not have been settled continue :  7
current path :  [0, 4, 2, 6]
start the next candidate :  3
There is a predecessor not have been settled continue :  3
current path :  [0, 4, 2, 6]
start the next candidate :  5
There is a predecessor not have been settled continue :  5
current path :  [0, 4, 2, 6]
start the next candidate :  7
There is a predecessor not have been settled continue :  7
current path :  [0, 4, 2]
start the next candidate :  7
There is a predecessor not have been settled continue :  7
current path :  [0, 4]
start the next candidate :  3
There is a predecessor not have been settled continue :  3
current path :  [0, 4]
start the next candidate :  5
There is a predecessor not have been settled continue :  5
current path :  [0, 4]
start the next c

In [69]:
filename

'42_symmetric.dat'

In [45]:
#def generate_predecessors(relations, extra_matrix, p_matrix, permutations):

relations = original_basis
zero_logic = generate_relations(relations, extra_matrix, p_matrix, permutations)
ret = dict()
for i, j in zero_logic:
    if i in ret:
        ret[i].add(j)
    else:
        ret[i] = set([j])

In [55]:
#def generate_relations(relations, extra_matrix, p_matrix, permutations):

p = p_matrix

l_h = []

for i,j in relations:
    current = (p_matrix[i] - p_matrix[j]).reshape((1,-1))
    l_h.append(current)

basis_matrix = np.concatenate(l_h,axis =0)

if len(extra_matrix) > 0:
    basis_matrix = np.concatenate([basis_matrix, extra_matrix],axis = 0)
    basis_matrix = np.unique(basis_matrix, axis = 0)

A_eq = basis_matrix.T
#b_ub = np.array([0 for _ in range(len(basis_matrix))])    
#c = np.array([0 for _ in range(len(basis_matrix))])
#A_ub = - np.identity(len(basis_matrix))

ret = set()
# print(basis_matrix)
for i,j in permutations:
    #print(i,j)
    m = Matrix(A_eq)
    y = (p[i]-p[j]).tolist()
    solvable = solvePLL(m,y)
    if solvable:
        ret.add((i,j))

In [53]:
extra_matrix

array([[-1,  1,  1, -1,  0,  0]])

In [54]:
basis_matrix

array([[-1,  1,  1, -1,  0,  0],
       [ 0,  0,  0,  0,  1, -1],
       [ 0,  0,  1, -1,  0,  0],
       [ 0,  1,  0, -1,  0,  0],
       [ 1, -1,  0,  0,  0,  0],
       [ 1,  0, -1,  0,  0,  0]])

In [61]:
y = (p[0]-p[1]).tolist()

In [62]:
y

[0, 0, 0, 0, -1, 1]

In [63]:
solvable = solvePLL(m,y)

In [64]:
solvable

True

In [65]:
#def solvePLL(m, y):
    
p = MixedIntegerLinearProgram(maximization=True, solver='ppl')
x = p.new_variable(nonnegative=True)
p.set_objective(0)
p.add_constraint(m * x <= y)

try:
    p.solve()
    return True
except MIPSolverException as err:
    s = str(err)
    if "no feasible solution" in s:
        return False
return False

SyntaxError: 'return' outside function (<ipython-input-65-abd704ce35f0>, line 10)